파이썬 파일에서 경로 오류로 인해 측정이 불가능한 경우, 해당 주피터 노트북으로 평가바랍니다.

In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf

# 현재 디렉토리의 utils 폴더 안에 위치시켜주세요.
# 꼭 classes.txt를 현재 디렉토리에 위치시켜주세요.
# tensorflow-addons 와 albumentations가 필요합니다.(모듈 임포트시 필요)
from utils.Network import Build_Network
from utils.SAM import train_step_sam
from utils.load_cifar100 import load_cifar100
from utils.Generator import DataGenerator
from utils.Metrices import recall, precision, f1score

network = Build_Network(100, booster=True)
model = tf.keras.Sequential()
model.add(network)
model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(100, activation='softmax'))

class MyModel(tf.keras.Model):
    def __init__(self,num_classes=100):
        super(MyModel, self).__init__(name='model')
        self.base=model
    def call(self, x):
        x = self.base(x)
        return x
    def train_step(self, data):
        return train_step_sam(self, data, rho=0.05)
    
model = MyModel(model)
model.build((None, 112, 112, 3))

model.summary()
model.load_weights('CIFAR100-weights.h5')

model.compile(
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy', precision, recall, f1score])

(x_train, y_train), (x_test, y_test) = load_cifar100()

x_test = x_test.astype('float32') / 255.0
y_test = tf.keras.utils.to_categorical(y_test, 100)

valid_data_generator = DataGenerator(x_test, y_test, augment=False)
_loss, _acc, _precision, _recall, _f1score = model.evaluate_generator(generator = valid_data_generator, verbose = 1)

print('loss: {:.4f}, accuracy: {:.4f}, precision: {:.4f}, recall: {:.4f}, f1score: {:.4f}'.format(_loss, _acc, _precision, _recall, _f1score))

c:\Users\KS_O_02\anaconda3\envs\tf\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 100)               4177664   
                                                                 
Total params: 4,177,664
Trainable params: 4,135,648
Non-trainable params: 42,016
_________________________________________________________________
{'apple': 0, 'aquarium_fish': 1, 'baby': 2, 'bear': 3, 'beaver': 4, 'bed': 5, 'bee': 6, 'beetle': 7, 'bicycle': 8, 'bottle': 9, 'bowl': 10, 'boy': 11, 'bridge': 12, 'bus': 13, 'butterfly': 14, 'camel': 15, 'can': 16, 'castle': 17, 'caterpillar': 18, 'cattle': 19, 'chair': 20, 'chimpanzee': 21, 'clock': 22, 'cloud': 23, 'cockroach': 24, 'couch': 25, 'crab': 26, 'crocodile': 27, 'cup': 28, 'dinosaur': 29, 'dolphin': 30, 'elephant': 31, 'flatfish': 32, 'forest': 33, 'fox': 34, 'girl': 35, 'hamster': 36, 'house': 37, 'kangaroo': 38, 'keyboard': 39, 'lamp': 40, 'lawn_mower

C:\Users\KS_O_02\AppData\Local\Temp\ipykernel_20028\2976164366.py:47: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  _loss, _acc, _precision, _recall, _f1score = model.evaluate_generator(generator = valid_data_generator, verbose = 1)


78/78 [==============================] - 42s 499ms/step - loss: 0.6482 - accuracy: 0.8162 - precision: 0.8542 - recall: 0.7937 - f1score: 0.8227
loss: 0.6482, accuracy: 0.8162, precision: 0.8542, recall: 0.7937, f1score: 0.8227
